# User Defined Functions

If you can use the built-in functions in `sql.functions`, you certainly should use them as they less likely to have bugs and can be optimized by Catalyst.

However, the built-in functions don't contain everything you need, and sometimes you have to write custom functions, known as User Defined Functions.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:</br>

* Create User Defined Functions (UDFs)
* Articulate performance advantages of Vectorized UDFs in Python

In [0]:
%run "./Includes/Classroom-Setup"

In [0]:
filePath = "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv"
rawDF = spark.read.csv(filePath, header=True, inferSchema=True, multiLine=True, escape='"')
display(rawDF)

Let's select the columns`id`, `host_name`, `bedrooms`, `neighbourhood_cleansed`, and `price`, as well as filter out all the rows that contain nulls.

In [0]:
airbnbDF = rawDF.select("id", "host_name", "bedrooms", "neighbourhood_cleansed", "price").dropna()
display(airbnbDF)

### User Defined Functions

We've seen many built-in functions (e.g. `avg`, `lit`, `col`, etc.). However, sometimes you might need a specific function that is not provided, so let's look at how to define your own **User Defined Function**.

For example, let's say we want to get the first initial from our `host_name` field. Let's start by writing that function in local Python/Scala.

In [0]:
def firstInitialFunction(name):
  return name[0]

firstInitialFunction("Jane")

Now we have to define a UDF that wraps the function. This will serialize our function and send it to the executors so that we can use it in our DataFrame.

In [0]:
firstInitialUDF = udf(firstInitialFunction)

In [0]:
from pyspark.sql.functions import col
display(airbnbDF.select(firstInitialUDF(col("host_name"))))

We can also create a UDF using `spark.sql.register`, which will create the UDF in the SQL namespace.

In [0]:
airbnbDF.createOrReplaceTempView("airbnbDF")

spark.udf.register("sql_udf", firstInitialFunction)

In [0]:
%sql
select sql_udf(host_name) as firstInitial from airbnbDF

### Decorator Syntax [Python Only]

Alternatively, you can define a UDF using decorator syntax in Python with the dataType the function will return. 

However, you cannot call the local Python function anymore (e.g. `decoratorUDF("Jane")` will not work)

In [0]:
%python
# Our input/output is a sting
@udf("string")
def decoratorUDF(name):
  return name[0]

In [0]:
display(airbnbDF.select(decoratorUDF(col("host_name"))))


UDFs provide even more functionality, but it is best to use a built in function wherever possible.

**UDF Drawbacks:**
* UDFs cannot be optimized by the Catalyst Optimizer
* The function **has to be serialized** and sent out to the executors
* In the case of Python, there is even more overhead - we have to **spin up a Python interpreter** on every Executor to run the UDF (e.g. Python UDFs much slower than Scala UDFs)

### Vectorized UDF

As of Spark 2.3, there are Vectorized UDFs available in Python to help speed up the computation.

* [Blog post](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html)
* [Documentation](https://spark.apache.org/docs/latest/sql-programming-guide.html#pyspark-usage-guide-for-pandas-with-apache-arrow)

![Benchmark](https://databricks.com/wp-content/uploads/2017/10/image1-4.png)

Vectorized UDFs utilize Apache Arrow to speed up computation. Let's see how that helps improve our processing time.

[Apache Arrow](https://arrow.apache.org/), is an in-memory columnar data format that is used in Spark to efficiently transfer data between JVM and Python processes. See more [here](https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html).

In [0]:
%python
from pyspark.sql.functions import pandas_udf

# We have a string input/output
@pandas_udf("string")
def vectorizedUDF(name):
  return name.str[0]

In [0]:
display(airbnbDF.select(vectorizedUDF(col("host_name"))))

We can also register these Vectorized UDFs to the SQL namespace.

In [0]:
%python
spark.udf.register("sql_vectorized_udf", vectorizedUDF)